In [1]:
libraries = c("dplyr","magrittr","tidyr","ggplot2","rstan","readxl","zoo","lubridate")
for(x in libraries) { library(x,character.only=TRUE,warn.conflicts=FALSE,quietly=TRUE) }

base_sz = 12 # base_size parameter
theme_set(theme_bw())

'%&%' = function(x,y) paste0(x,y)

options(mc.cores = parallel::detectCores())
rstan_options(auto_write = TRUE)

packageVersion("rstan")
packageVersion("StanHeaders")
rstan::stan_version()

rstan (Version 2.19.3, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)



[1] ‘2.19.3’

[1] ‘2.21.0.1’

[1] "2.21.0"

In [7]:
filenames = c("data_incper", "dthdata_hosp_dth", "dthdata_ons_hosp", "dthdata_ons_dth", "data_ons_hosp", "data_incper_inclwuhan")

# <font color="purple">Without truncation for all datasets except *data_incper_inclwuhan* dataset</font>

## <font color="orange">Lognormal distribution</font>

In [16]:
for (idx in 1:(length(filenames)-1)) 
{
    ## main dir for Stan simulations
    standirname = 'stan-sims/'%&%filenames[idx]%&%"-lognormal-truncated"
    unlink(standirname, recursive=T)
    dir.create(standirname)

    datafilename = "../../data/"%&%filenames[idx]%&%".csv"
    read.table(datafilename, sep=",", header=T) %>% select(EL,ER,SL,SR,tstar) %>% mutate(dist = (SR+SL)/2-(ER+EL)/2) -> df

    # Dumping data
    N = nrow(df)
    E_L = df$EL
    E_R = df$ER
    S_L = df$SL
    S_R = df$SR
    r = 0.14
    upper_bound = df$tstar[1]
    stan_rdump(c('E_L', 'E_R', 'S_L', 'S_R', 'N', 'r', 'upper_bound'), file=standirname%&%"/Data.R") 

    # Dumping initial conditions
    e_raw = rep(.4, N)
    s_raw = rep(.6, N)
    logmean_SI = log(mean(df$dist))
    logsd_SI = log(sd(df$dist))
    stan_rdump(c('e_raw', 's_raw', 'logmean_SI', 'logsd_SI'), file=standirname%&%"/Init.R") 

    # Stan program
    "functions {
        real[] fstar_ode(real t, real[] z, real[] theta, data real[] x_r, int[] x_i) {
            int N = x_i[1]; // number of records

            real e[N] = theta[1:N];
            real param1 = theta[N+1];
            real param2 = theta[N+2];

            real upper_bound = x_r[1];
            real r = x_r[2];

            real dzdt[N];
            real tstar[N];

            for (k in 1:N) {
                tstar[k] = upper_bound - e[k];
                dzdt[k] = exp(lognormal_lcdf(tstar[k]*(1.0-t) | param1, param2)) * r * tstar[k] * exp(-r*tstar[k]*t) / (1.0 - exp(-r*tstar[k]*t));
            }

            return dzdt;
        }
    }

    data {
        int<lower = 0> N; // number of records
        vector<lower = 0>[N] E_L;
        vector<lower = 0>[N] E_R;
        vector<lower = 0>[N] S_L;
        vector<lower = 0>[N] S_R;
        real<lower = 0> upper_bound;
        real<lower = 0> r;
    }

    transformed data {
        int X_i[1] = {N};

        real X_r[2] = {upper_bound, r};
    }

    parameters {
        real logmean_SI;
        real logsd_SI;

        vector<lower = 0, upper = 1>[N] e_raw;
        vector<lower = 0, upper = 1>[N] s_raw;
    }

    transformed parameters {
        real<lower = 0> param2 = sqrt(log((exp(2*(logsd_SI-logmean_SI))+1.0)));
        real param1 = logmean_SI - param2^2/2.0;

        vector<lower = min(E_L), upper = max(E_R)>[N] e;
        vector<lower = min(S_L), upper = max(S_R)>[N] s;

        real Z[N]; 

        {
            real theta[N+2];
            real Z0[N];

            s = S_L + (S_R - S_L) .* s_raw;
            for (k in 1:N) {
                if (E_R[k] > s[k]) 
                    e[k] = E_L[k] + (s[k] - E_L[k]) * e_raw[k];
                else
                    e[k] = E_L[k] + (E_R[k] - E_L[k]) * e_raw[k];
            }

            for (k in 1:N) {
                Z0[k] = 0.0;
                theta[k] = e[k];
            }
            theta[N+1] = param1;
            theta[N+2] = param2;

            Z = to_array_1d(integrate_ode_rk45(fstar_ode, Z0, 0.001, {1.0}, theta, X_r, X_i, 1e-5, 1e-3, 5e2));
        }
    }

    model {
        logmean_SI ~ std_normal();
        logsd_SI ~ std_normal();

        e_raw ~ normal(0.5, 1.0);
        s_raw ~ normal(0.5, 1.0);

        for (k in 1:N) 
            target += lognormal_lpdf(s[k] - e[k] | param1, param2) - log(Z[k]);
    }

    generated quantities {
        real<lower = 0> mean_SI = exp(param1 + param2^2/2);
        real<lower = 0> sd_SI = sqrt((exp(param2^2)-1)*exp(2*param1+param2^2));

        vector[N] log_likelihood;
        for (k in 1:N)
            log_likelihood[k] = lognormal_lpdf(s[k] - e[k] | param1, param2) - log(Z[k]);
    }" %>% cat(file=standirname %&% "/fit.stan", sep="", fill=TRUE)

    standistribdir = "../../../../CmdStan"
    stanscriptdir = "../Dropbox/Hokkaido_Wuhan_IncubationPeriod_2020/scripts/Andrei-more_accurate/"%&%standirname
    ## bash file
    "#!/bin/bash
    cwd=$(pwd)
    cd \""%&%standistribdir%&%"\"
    make -j6 \""%&%stanscriptdir%&%"/fit\"
    cd \""%&%stanscriptdir%&%"\"
    mkdir -p diagnostics
    for i in {1..10}
    do
        echo Running ${i}
        SEEDNUMBER=$((1+$i))
        ./fit \\
            method=sample num_samples=10000 num_warmup=10000 save_warmup=0 \\
                adapt delta=0.98 \\
                algorithm=hmc \\
                    engine=nuts \\
            random seed=${SEEDNUMBER} \\
            id=$i \\
            data file=Data.R \\
            init=Init.R \\
            output file=trace-$i.csv \\
                diagnostic_file=diagnostics/diagnostics-$i.csv > diagnostics/output-$i.txt &
    done
    echo Finished sampling haha!
    " %>% cat(file=standirname%&%"/fit.sh", sep="", fill=TRUE)

    ## running the bash script
    system("bash "%&%standirname%&%"/fit.sh", intern = TRUE)
}

[1] ""                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
 [2] "--- Translating Stan model to C++ code ---"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
 [3] "bin/stanc  --o=../Dropbox/Hokkaido_Wuhan_IncubationPeriod_2020/scripts/Andr

<font color="red">For the dataset **data_ons_hosp**, the inference with using ODE was very slow, so we switched to the integral instead</font>

In [10]:
idx = length(filenames)-2
'stan-sims/'%&%filenames[idx]%&%"-lognormal-truncated"

[1] "stan-sims/dthdata_ons_dth-lognormal-truncated"

In [18]:
for (idx in 1:(length(filenames)-1)) 
{
    ## main dir for Stan simulations
    standirname = 'stan-sims/'%&%filenames[idx]%&%"-lognormal-truncated"
    unlink(standirname, recursive=T)
    dir.create(standirname)
    print(standirname)

    datafilename = "../../data/"%&%filenames[idx]%&%".csv"
    read.table(datafilename, sep=",", header=T) %>% select(EL,ER,SL,SR,tstar) %>% mutate(dist = (SR+SL)/2-(ER+EL)/2) -> df

    # Dumping data
    N = nrow(df)
    E_L = df$EL
    E_R = df$ER
    S_L = df$SL
    S_R = df$SR
    r = 0.14
    upper_bound = df$tstar[1]
    stan_rdump(c('E_L', 'E_R', 'S_L', 'S_R', 'N', 'r', 'upper_bound'), file=standirname%&%"/Data.R") 

    # Dumping initial conditions
    e_raw = rep(.4, N)
    s_raw = rep(.6, N)
    logmean_SI = log(mean(df$dist))
    logsd_SI = log(sd(df$dist))
    stan_rdump(c('e_raw', 's_raw', 'logmean_SI', 'logsd_SI'), file=standirname%&%"/Init.R") 

    # Stan program
    "functions {
            real fstar(real x,          // Function argument
                       real xc,         // Complement of function argument on the domain (defined later)
                       real[] theta,    // parameters
                       real[] x_r,      // data (real)
                       int[] x_i) {     // data (integer)

                real param1 = theta[1];
                real param2 = theta[2];
                real tstar = theta[3];
                real r = x_r[1];

                return exp(lognormal_lcdf(tstar-x | param1, param2)) * r * exp(-r*x) / (1.0 - exp(-r*x));
            }
        }

        data {
            int<lower = 0> N; // number of records
            vector<lower = 0>[N] E_L;
            vector<lower = 0>[N] E_R;
            vector<lower = 0>[N] S_L;
            vector<lower = 0>[N] S_R;
            real<lower = 0> upper_bound;
            real<lower = 0> r;
        }

        transformed data {
            int X_i[0]; //empty array
        }

        parameters {
            real logmean_SI;
            real logsd_SI;

            vector<lower = 0, upper = 1>[N] E_raw;
            vector<lower = 0, upper = 1>[N] S_raw;
        }

        transformed parameters {
            real<lower = 0> param2 = sqrt(log((exp(2*(logsd_SI-logmean_SI))+1.0)));
            real param1 = logmean_SI - param2^2/2.0;

            vector<lower = min(E_L), upper = max(E_R)>[N] E;
            vector<lower = min(S_L), upper = max(S_R)>[N] S;
            vector<lower = 0>[N] t;
            vector<lower = 0>[N] tstar;

            E = E_L + (E_R - E_L) .* E_raw;
            for (k in 1:N) {
                if (E[k] > S_L[k]) 
                    S[k] = E[k] + (S_R[k] - E[k]) * S_raw[k];
                else 
                    S[k] = S_L[k] + (S_R[k] - S_L[k]) * S_raw[k];
            }
            t = S - E;
            tstar = upper_bound - E;
        }

        model {
            logmean_SI ~ std_normal();
            logsd_SI ~ std_normal();

            E_raw ~ normal(0.5, 1.0);
            S_raw ~ normal(0.5, 1.0);

            for (k in 1:N) 
                target += lognormal_lpdf(S[k] - E[k] | param1, param2)
                            - log(integrate_1d(fstar, 0.001, tstar[k] - 0.001, {param1, param2, tstar[k]}, {r}, X_i));
        }

        generated quantities {
            real<lower = 0> mean_SI = exp(logmean_SI);
            real<lower = 0> sd_SI = exp(logsd_SI);
        }
" %>% cat(file=standirname %&% "/fit.stan", sep="", fill=TRUE)

    standistribdir = "../../../../CmdStan"
    stanscriptdir = "../Dropbox/Hokkaido_Wuhan_IncubationPeriod_2020/scripts/Andrei-more_accurate/"%&%standirname
    ## bash file
    "#!/bin/bash
    cwd=$(pwd)
    cd \""%&%standistribdir%&%"\"
    make -j6 \""%&%stanscriptdir%&%"/fit\"
    cd \""%&%stanscriptdir%&%"\"
    mkdir -p diagnostics
    for i in {1..10}
    do
        echo Running ${i}
        SEEDNUMBER=$((1+$i))
        ./fit \\
            method=sample num_samples=10000 num_warmup=10000 save_warmup=0 \\
                adapt delta=0.98 \\
                algorithm=hmc \\
                    engine=nuts \\
            random seed=${SEEDNUMBER} \\
            id=$i \\
            data file=Data.R \\
            init=Init.R \\
            output file=trace-$i.csv \\
                diagnostic_file=diagnostics/diagnostics-$i.csv > diagnostics/output-$i.txt &
    done
    echo Finished sampling haha!
    " %>% cat(file=standirname%&%"/fit.sh", sep="", fill=TRUE)

    ## running the bash script
    system("bash "%&%standirname%&%"/fit.sh", intern = TRUE)
}

[1] "stan-sims/data_ons_hosp-lognormal-truncated"


[1] ""                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
 [2] "--- Translating Stan model to C++ code ---"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
 [3] "bin/stanc  --o=../Dropbox/Hokkaido_Wuhan_IncubationPeriod_2020/scripts/Andrei-more_accu

## <font color="orange">Gamma distribution</font> 

**NOT USED IN THE PAPER**

In [21]:
for (idx in 1:(length(filenames)-1))  #
{
    ## main dir for Stan simulations
    standirname = 'stan-sims/'%&%filenames[idx]%&%"-gamma-truncated"
    unlink(standirname, recursive=T)
    dir.create(standirname)

    datafilename = "../../data/"%&%filenames[idx]%&%".csv"
    read.table(datafilename, sep=",", header=T) %>% select(EL,ER,SL,SR,tstar) %>% mutate(dist = (SR+SL)/2-(ER+EL)/2) -> df

    # Dumping data
    N = nrow(df)
    E_L = df$EL
    E_R = df$ER
    S_L = df$SL
    S_R = df$SR
    r = 0.14
    upper_bound = df$tstar[1]
    stan_rdump(c('E_L', 'E_R', 'S_L', 'S_R', 'N', 'r', 'upper_bound'), file=standirname%&%"/Data.R") 

    # Dumping initial conditions
    e_raw = rep(.2, N)
    s_raw = rep(.8, N)
    mean_SI = mean(df$dist)
    sd_SI = sd(df$dist)
    stan_rdump(c('e_raw', 's_raw', 'mean_SI', 'sd_SI'), file=standirname%&%"/Init.R")  

    # Stan program
    "functions {
        real[] fstar_ode(real t, real[] z, real[] theta, data real[] x_r, int[] x_i) {
            int N = x_i[1]; // number of records

            real e[N] = theta[1:N];
            real param1 = theta[N+1];
            real param2 = theta[N+2];

            real upper_bound = x_r[1];
            real r = x_r[2];

            real dzdt[N];
            real tstar[N];

            for (k in 1:N) {
                tstar[k] = upper_bound - e[k];
                dzdt[k] = exp(gamma_lcdf(tstar[k]*(1.0-t) | param1, param2)) * r * tstar[k] * exp(-r*tstar[k]*t) / (1.0 - exp(-r*tstar[k]*t));
            }

            return dzdt;
        }
    }

    data {
        int<lower = 0> N; // number of records
        vector<lower = 0>[N] E_L;
        vector<lower = 0>[N] E_R;
        vector<lower = 0>[N] S_L;
        vector<lower = 0>[N] S_R;
        real<lower = 0> upper_bound;
        real<lower = 0> r;
    }

    transformed data {
        int X_i[1] = {N};

        real X_r[2] = {upper_bound, r};
    }

    parameters {
        real<lower=0> mean_SI;
        real<lower=0> sd_SI;

        vector<lower = 0, upper = 1>[N] e_raw;
        vector<lower = 0, upper = 1>[N] s_raw;
    }

    transformed parameters {
        real<lower = 0> param1 = (mean_SI/sd_SI)^2;
        real<lower = 0> param2 = mean_SI/(sd_SI^2);

        vector<lower = min(E_L), upper = max(E_R)>[N] e;
        vector<lower = min(S_L), upper = max(S_R)>[N] s;

        real Z[N]; 

        {
            real theta[N+2];
            real Z0[N];

            s = S_L + (S_R - S_L) .* s_raw;
            for (k in 1:N) {
                if (E_R[k] > s[k]) 
                    e[k] = E_L[k] + (s[k] - E_L[k]) * e_raw[k];
                else
                    e[k] = E_L[k] + (E_R[k] - E_L[k]) * e_raw[k];
            }

            for (k in 1:N) {
                Z0[k] = 0.0;
                theta[k] = e[k];
            }
            theta[N+1] = param1;
            theta[N+2] = param2;

            Z = to_array_1d(integrate_ode_rk45(fstar_ode, Z0, 0.001, {1.0}, theta, X_r, X_i, 1e-5, 1e-3, 5e2));
        }
    }

    model {
        mean_SI ~ normal(5.0, 10.0);
        sd_SI ~ cauchy(0, 5.0);

        e_raw ~ normal(0.5, 1.0);
        s_raw ~ normal(0.5, 1.0);

        for (k in 1:N) 
            target += gamma_lpdf(s[k] - e[k] | param1, param2) - log(Z[k]);
    }

    generated quantities {
        vector[N] log_likelihood;
        for (k in 1:N)
            log_likelihood[k] = gamma_lpdf(s[k] - e[k] | param1, param2) - log(Z[k]);
    }" %>% cat(file=standirname %&% "/fit.stan", sep="", fill=TRUE)

    standistribdir = "../../../../CmdStan"
    stanscriptdir = "../Dropbox/Hokkaido_Wuhan_IncubationPeriod_2020/scripts/Andrei-more_accurate/"%&%standirname
    ## bash file
    "#!/bin/bash
    cwd=$(pwd)
    cd \""%&%standistribdir%&%"\"
    make -j6 \""%&%stanscriptdir%&%"/fit\"
    cd \""%&%stanscriptdir%&%"\"
    mkdir -p diagnostics
    for i in {1..5}
    do
        echo Running ${i}
        SEEDNUMBER=$((1+$i))
        ./fit \\
            method=sample num_samples=20000 num_warmup=10000 save_warmup=0 \\
                adapt delta=0.98 \\
                algorithm=hmc \\
                    engine=nuts \\
            random seed=${SEEDNUMBER} \\
            id=$i \\
            data file=Data.R \\
            init=Init.R \\
            output file=trace-$i.csv \\
                diagnostic_file=diagnostics/diagnostics-$i.csv > diagnostics/output-$i.txt &
    done
    echo Finished sampling haha!
    " %>% cat(file=standirname%&%"/fit.sh", sep="", fill=TRUE)

    ## running the bash script
    system("bash "%&%standirname%&%"/fit.sh", intern = TRUE)
}

## <font color="orange">Weibull distribution</font>

**NOT USED IN THE PAPER**

In [26]:
for (idx in 1:(length(filenames)-1))  #
{
    ## main dir for Stan simulations
    standirname = 'stan-sims/'%&%filenames[idx]%&%"-weibull-truncated"
    unlink(standirname, recursive=T)
    dir.create(standirname)

    datafilename = "../../data/"%&%filenames[idx]%&%".csv"
    read.table(datafilename, sep=",", header=T) %>% select(EL,ER,SL,SR,tstar) %>% mutate(dist = (SR+SL)/2-(ER+EL)/2) -> df

    # Dumping data
    N = nrow(df)
    E_L = df$EL
    E_R = df$ER
    S_L = df$SL
    S_R = df$SR
    r = 0.14
    upper_bound = df$tstar[1]
    stan_rdump(c('E_L', 'E_R', 'S_L', 'S_R', 'N', 'r', 'upper_bound'), file=standirname%&%"/Data.R") 

    # Dumping initial conditions
    e_raw = rep(.4, N)
    s_raw = rep(.6, N)
    mean_SI = mean(df$dist)
    param1 = 1.75
    stan_rdump(c('e_raw', 's_raw', 'mean_SI', 'param1'), file=standirname%&%"/Init.R")  

    # Stan program
    "functions {
        real[] fstar_ode(real t, real[] z, real[] theta, data real[] x_r, int[] x_i) {
            int N = x_i[1]; // number of records

            real e[N] = theta[1:N];
            real param1 = theta[N+1];
            real param2 = theta[N+2];

            real upper_bound = x_r[1];
            real r = x_r[2];

            real dzdt[N];
            real tstar[N];

            for (k in 1:N) {
                tstar[k] = upper_bound - e[k];
                dzdt[k] = - expm1(-((1.0-t)*tstar[k]/param2)^param1) * r * tstar[k] * exp(-r*tstar[k]*t) / (1.0 - exp(-r*tstar[k]*t));
            }

            return dzdt;
        }
    }

    data {
        int<lower = 0> N; // number of records
        vector<lower = 0>[N] E_L;
        vector<lower = 0>[N] E_R;
        vector<lower = 0>[N] S_L;
        vector<lower = 0>[N] S_R;
        real<lower = 0> upper_bound;
        real<lower = 0> r;
    }

    transformed data {
        int X_i[1] = {N};

        real X_r[2] = {upper_bound, r};
    }

    parameters {
        real<lower = 0> mean_SI;
        real<lower = 0> param1;

        vector<lower = 0, upper = 1>[N] e_raw;
        vector<lower = 0, upper = 1>[N] s_raw;
    }

    transformed parameters {
        real<lower = 0> param2 = mean_SI/tgamma(1.0+1.0/param1);

        vector<lower = min(E_L), upper = max(E_R)>[N] e;
        vector<lower = min(S_L), upper = max(S_R)>[N] s;

        real Z[N]; 

        {
            real theta[N+2];
            real Z0[N];

            s = S_L + (S_R - S_L) .* s_raw;
            for (k in 1:N) {
                if (E_R[k] > s[k]) 
                    e[k] = E_L[k] + (s[k] - E_L[k]) * e_raw[k];
                else
                    e[k] = E_L[k] + (E_R[k] - E_L[k]) * e_raw[k];
            }

            for (k in 1:N) {
                Z0[k] = 0.0;
                theta[k] = e[k];
            }
            theta[N+1] = param1;
            theta[N+2] = param2;

            Z = to_array_1d(integrate_ode_rk45(fstar_ode, Z0, 0.001, {1.0}, theta, X_r, X_i, 1e-5, 1e-3, 5e2));
        }
    }

    model {
        mean_SI ~ normal(5.0, 10.0);
        param1 ~ exponential(0.0001);

        e_raw ~ normal(0.5, 1.0);
        s_raw ~ normal(0.5, 1.0);

        for (k in 1:N) 
            target += weibull_lpdf(s[k] - e[k] | param1, param2) - log(Z[k]);
    }

    generated quantities {
        vector[N] log_likelihood;
        for (k in 1:N)
            log_likelihood[k] = weibull_lpdf(s[k] - e[k] | param1, param2) - log(Z[k]);
    }" %>% cat(file=standirname %&% "/fit.stan", sep="", fill=TRUE)

    standistribdir = "../../../../CmdStan"
    stanscriptdir = "../Dropbox/Hokkaido_Wuhan_IncubationPeriod_2020/scripts/Andrei-more_accurate/"%&%standirname
    ## bash file
    "#!/bin/bash
    cwd=$(pwd)
    cd \""%&%standistribdir%&%"\"
    make -j6 \""%&%stanscriptdir%&%"/fit\"
    cd \""%&%stanscriptdir%&%"\"
    mkdir -p diagnostics
    for i in {1..10}
    do
        echo Running ${i}
        SEEDNUMBER=$((1+$i))
        ./fit \\
            method=sample num_samples=10000 num_warmup=10000 save_warmup=0 \\
                adapt delta=0.98 \\
                algorithm=hmc \\
                    engine=nuts \\
            random seed=${SEEDNUMBER} \\
            id=$i \\
            data file=Data.R \\
            init=Init.R \\
            output file=trace-$i.csv \\
                diagnostic_file=diagnostics/diagnostics-$i.csv > diagnostics/output-$i.txt &
    done
    echo Finished sampling haha!
    " %>% cat(file=standirname%&%"/fit.sh", sep="", fill=TRUE)

    ## running the bash script
    system("bash "%&%standirname%&%"/fit.sh", intern = TRUE)
}

[1] ""                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
 [2] "--- Translating Stan model to C++ code ---"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [3] "bin/stanc  --o=../Dropbox/Hokkaido_Wuhan_IncubationPeriod_2020/scripts/Andrei-more_